In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


In [2]:
dataset_path = r"E:\OneDrive\Desktop\CLICK\Amrita\SEM 5\PROJECTS\Project_comp_sec\Ransomware_headers.csv"   
df = pd.read_csv(dataset_path)

In [3]:
df.head(5)

,ID,filename,GR,family,0,1,2,3,4,5,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,10000,2XClient_12.0_build_2193.paf.exe,0,0,77,90,144,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,10001,7-ZipPortable_21.07.paf.exe,0,0,77,90,144,0,3,0,...,0,0,0,0,0,0,0,0,0,0
2,10002,ARP.EXE,0,0,77,90,144,0,3,0,...,0,0,0,0,0,0,0,0,0,0
3,10003,AccountsControlHost.exe,0,0,77,90,144,0,3,0,...,0,0,0,0,0,0,0,0,0,0
4,10004,ActionUriServer.exe,0,0,77,90,144,0,3,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Select all columns except the 1st, 2nd, 4th as features
X = df.drop(df.columns[[0,1,3]], axis=1)

# Select the 3rd column (index 2) as the target
y = df.iloc[:, 2]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Standardize features (mean=0, variance=1)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [7]:
print('hey')

hey


LR without pre

In [8]:
from sklearn.linear_model import LogisticRegression
# Logistic Regression Model
lr = LogisticRegression(max_iter=1000, random_state = 53, verbose=3)
lr.fit(X_train_scaled, y_train)


y_pred_lr = lr.predict(X_test_scaled)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))


Logistic Regression Accuracy: 0.9930555555555556
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       222
           1       0.99      1.00      0.99       210

    accuracy                           0.99       432
   macro avg       0.99      0.99      0.99       432
weighted avg       0.99      0.99      0.99       432



Ensemble learning classifier (without pre)

In [9]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier  # XGBoost for classification
from sklearn.metrics import accuracy_score, classification_report

# Define individual classifiers
lr = LogisticRegression(max_iter=1000)
rf = RandomForestClassifier()
xgb = XGBClassifier(eval_metric='mlogloss')  # XGBoost configuration

# Create an ensemble model using Logistic Regression, Random Forest, and XGBoost
voting_clf = VotingClassifier(estimators=[
    ('lr', lr), ('rf', rf), ('xgb', xgb)], voting='hard')

# Train the ensemble model
voting_clf.fit(X_train, y_train)

# Predictions and evaluation
y_pred_voting = voting_clf.predict(X_test)
print("Ensemble Voting Classifier Accuracy:", accuracy_score(y_test, y_pred_voting))
print(classification_report(y_test, y_pred_voting))


Ensemble Voting Classifier Accuracy: 0.9976851851851852
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       222
           1       1.00      1.00      1.00       210

    accuracy                           1.00       432
   macro avg       1.00      1.00      1.00       432
weighted avg       1.00      1.00      1.00       432



In [10]:
print('Hey')

Hey


LR usinf rfe

In [11]:
from sklearn.feature_selection import RFE

# Use Logistic Regression to rank feature importance
lr = LogisticRegression(max_iter=1000)
rfe = RFE(estimator=lr, n_features_to_select=100)  # Choose 10 features
X_train_rfe = rfe.fit_transform(X_train_scaled, y_train)
X_test_rfe = rfe.transform(X_test_scaled)

# Fit Logistic Regression on the reduced feature set
lr.fit(X_train_rfe, y_train)
y_pred_rfe = lr.predict(X_test_rfe)

# Evaluate accuracy
print("RFE Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_rfe))


RFE Logistic Regression Accuracy: 0.9976851851851852


Applying PCA fir dimensionality reduct (reduced to 500)

In [12]:
from sklearn.decomposition import PCA

pca = PCA(n_components=400)  
X_pca = pca.fit_transform(X)

# Step 4: Explained variance ratio (to see how much variance is captured by each component)
explained_variance = pca.explained_variance_ratio_
print("Explained Variance Ratio by each Principal Component:", explained_variance)
print("Total Explained Variance:", np.sum(explained_variance))

Explained Variance Ratio by each Principal Component: [1.41415340e-01 6.75026127e-02 6.33037443e-02 5.73638364e-02
 4.76375520e-02 4.41015521e-02 3.68754793e-02 3.50506769e-02
 3.09994263e-02 2.85965949e-02 2.64507785e-02 2.09949749e-02
 1.81402322e-02 1.47806042e-02 1.35376259e-02 1.30094049e-02
 1.22903947e-02 1.01940954e-02 8.98086459e-03 7.89576299e-03
 7.48486292e-03 7.06756644e-03 6.94229654e-03 6.10970484e-03
 5.96193254e-03 5.78609582e-03 5.59549140e-03 5.18549227e-03
 4.91869005e-03 4.76215366e-03 4.67642612e-03 4.42116159e-03
 4.22052125e-03 4.05629078e-03 3.94870132e-03 3.80479865e-03
 3.65577283e-03 3.54398580e-03 3.40450721e-03 3.19660585e-03
 3.06429903e-03 2.96745552e-03 2.91352935e-03 2.72488353e-03
 2.70406399e-03 2.57008724e-03 2.50335940e-03 2.43561543e-03
 2.39918647e-03 2.32123457e-03 2.26262973e-03 2.19699688e-03
 2.16656695e-03 2.13624546e-03 2.11361369e-03 2.04858497e-03
 1.99493469e-03 1.94551603e-03 1.91994915e-03 1.86033025e-03
 1.85371132e-03 1.78937518e-03 

rfc after pca

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

rfc = RandomForestClassifier(n_estimators=200, max_depth=None, min_samples_split=2, random_state=42)    

rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

Test Accuracy: 0.9004629629629629
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.99      0.91       222
           1       0.98      0.81      0.89       210

    accuracy                           0.90       432
   macro avg       0.91      0.90      0.90       432
weighted avg       0.91      0.90      0.90       432



Ensemble model after applying pca

In [14]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier  # XGBoost for classification
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Define individual classifiers
lr = LogisticRegression(max_iter=1000)
rf = RandomForestClassifier()
xgb = XGBClassifier(eval_metric='mlogloss')  # XGBoost configuration

# Create an ensemble model using Logistic Regression, Random Forest, and XGBoost
voting_clf = VotingClassifier(estimators=[
    ('lr', lr), ('rf', rf), ('xgb', xgb)], voting='hard')

# Train the ensemble model
voting_clf.fit(X_train, y_train)

# Predictions and evaluation
y_pred_voting = voting_clf.predict(X_test)
print("Ensemble Voting Classifier Accuracy:", accuracy_score(y_test, y_pred_voting))
print(classification_report(y_test, y_pred_voting))


Ensemble Voting Classifier Accuracy: 0.9375
              precision    recall  f1-score   support

           0       0.91      0.97      0.94       222
           1       0.96      0.90      0.93       210

    accuracy                           0.94       432
   macro avg       0.94      0.94      0.94       432
weighted avg       0.94      0.94      0.94       432

